In [1]:
import glob
import pandas as pd
import pickle
import ast

In [2]:
# Grab all files witin data folder for each month in 2019
extension = 'csv'
directories = ['2019-01-01', '2019-02-01', '2019-03-01', '2019-04-01', '2019-05-01', '2019-06-01',
              '2019-07-01', '2019-08-01', '2019-09-01', '2019-10-01', '2019-11-01', '2019-12-01']

all_files_lists = []
for folder in directories:
    file_names = [i for i in glob.glob('data/Kickstarter_{}/*.{}'.format(folder, extension))]
    all_files_lists.append(file_names)

In [3]:
# Flatten the resulting list to create a list of all file paths
all_files = [item for sublist in all_files_lists for item in sublist]

# Dataframe concatenation and loading

In [4]:
def load_data_into_dataframe(file_names):
    """
    Takes in a list of file names and creates dataframes from each element. Then each dataframe is filtered and
    cleaned, then all dataframes are concatenated along columns.

    :param file_names (list): List of file names as strings
    :return: Pandas dataframe
    """

    # Set dataframe filters and parameters
    columns_to_keep = ['backers_count', 'creator', 'category', 'converted_pledged_amount', 'country',
        'created_at', 'deadline', 'goal', 'id', 'launched_at', 'name', 'pledged', 'source_url', 'spotlight',
        'staff_pick', 'state', 'state_changed_at', 'urls', 'usd_pledged']
    
    data_frames = []
    for file in file_names:
        df = pd.read_csv(file)

        # Filter based on specified criteria
        df = df.loc[:, columns_to_keep]

        # Convert dates from unix to datetime
        date_cols = ['created_at', 'deadline', 'launched_at', 'state_changed_at']
        for col in date_cols:
            df[col] = pd.to_datetime(df[col], origin='unix', unit='s')

        # Clean rows
        df.drop_duplicates('name', keep=False, inplace=True)

        data_frames.append(df)
        
    combined_df = pd.concat(data_frames)
    combined_df.reset_index(inplace=True, drop=True)
    return combined_df

In [5]:
df = load_data_into_dataframe(all_files)

In [6]:
print(df.shape)
df.head()

(2512270, 19)


,backers_count,creator,category,converted_pledged_amount,country,created_at,deadline,goal,id,launched_at,name,pledged,source_url,spotlight,staff_pick,state,state_changed_at,urls,usd_pledged
0,1,"{""id"":983022919,""name"":""Rhys Kucharski"",""is_re...","{""id"":356,""name"":""Woodworking"",""slug"":""crafts/...",240,US,2016-09-11 22:05:51,2016-12-05 19:42:23,5000.0,1504859185,2016-11-05 18:42:23,Industrial Bamboo Table,240.0,https://www.kickstarter.com/discover/categorie...,False,False,failed,2016-12-05 19:42:23,"{""web"":{""project"":""https://www.kickstarter.com...",240.000000
1,3,"{""id"":101531536,""name"":""Picadoo Custom Art"",""i...","{""id"":23,""name"":""Painting"",""slug"":""art/paintin...",243,CA,2015-07-10 14:59:32,2015-08-24 12:00:34,1000.0,49266114,2015-07-21 12:00:34,"Custom Pet Portraits on Canvas- ""From Pixels t...",322.0,https://www.kickstarter.com/discover/categorie...,False,False,failed,2015-08-24 12:00:34,"{""web"":{""project"":""https://www.kickstarter.com...",247.950175
2,243,"{""id"":372111659,""name"":""Kevin, Domingo & Suzie...","{""id"":311,""name"":""Food Trucks"",""slug"":""food/fo...",41738,US,2015-03-24 17:41:14,2015-05-15 16:22:34,35000.0,1228074690,2015-04-15 16:22:34,The Barmobile: Boston's Mobile Cocktail Cateri...,41738.0,https://www.kickstarter.com/discover/categorie...,True,True,successful,2015-05-15 16:22:34,"{""web"":{""project"":""https://www.kickstarter.com...",41738.000000
3,27,"{""id"":1336552462,""name"":""Christine Almstrom"",""...","{""id"":46,""name"":""Children's Books"",""slug"":""pub...",3115,US,2017-05-18 12:30:32,2017-07-16 15:03:03,3000.0,330962986,2017-06-01 15:03:03,Grandfather Thunder & The Night Horses,3115.0,https://www.kickstarter.com/discover/categorie...,True,False,successful,2017-07-16 15:03:04,"{""web"":{""project"":""https://www.kickstarter.com...",3115.000000
4,3,"{""id"":2032403632,""name"":""David Russell"",""is_re...","{""id"":298,""name"":""Movie Theaters"",""slug"":""film...",674,IE,2015-12-14 19:38:41,2016-02-13 01:56:30,15000.0,1657821447,2015-12-15 01:56:30,Save Me-A film to hightlight depression (Cance...,601.0,https://www.kickstarter.com/discover/categorie...,False,False,canceled,2016-02-10 00:54:26,"{""web"":{""project"":""https://www.kickstarter.com...",660.680598


## Include only completed projects

In [7]:
df = df.loc[df['state'].isin(['successful', 'failed'])]

## Cleaning columns read in as JSON

In [8]:
# Convert all 'category' and 'urls' JSON columns to dictionaries
for col in ['category', 'urls']:
    df[col] = df[col].apply(ast.literal_eval)

In [9]:
def split_dict_columns(row):
    """
    Takes in a dictionary and returns the value at specific keys. this function deals with extracting
    the category names and project urls from converted JSON columns

    :param row (dict): A standard dictionary
    :return: Any object
    """

    if 'slug' in row.keys():
        return row['slug']
    elif 'web' in row.keys():
        return row['web']['project']

In [10]:
# Create a new column for categories
category_lists = df['category'].apply(split_dict_columns).map(lambda x: x.split('/'))
df['new_category'] = category_lists.map(lambda x: x[0])
df['new_category']

0              crafts
1                 art
2                food
3          publishing
5                food
              ...    
2512264        design
2512265          food
2512266         dance
2512267          food
2512268    publishing
Name: new_category, Length: 2321752, dtype: object

In [11]:
# Create a new column for subcategories
df['subcategory'] = category_lists.map(lambda x: x[-1])
df['subcategory']

0               woodworking
1                  painting
2               food trucks
3          children's books
5                    drinks
                 ...       
2512264        architecture
2512265              drinks
2512266               dance
2512267              drinks
2512268          nonfiction
Name: subcategory, Length: 2321752, dtype: object

In [12]:
# Create a new column for project urls
df['project_url'] = df['urls'].apply(split_dict_columns)
df['project_url']

0          https://www.kickstarter.com/projects/983022919...
1          https://www.kickstarter.com/projects/101531536...
2          https://www.kickstarter.com/projects/372111659...
3          https://www.kickstarter.com/projects/133655246...
5          https://www.kickstarter.com/projects/107038669...
                                 ...                        
2512264    https://www.kickstarter.com/projects/862442095...
2512265    https://www.kickstarter.com/projects/128955704...
2512266    https://www.kickstarter.com/projects/126176310...
2512267    https://www.kickstarter.com/projects/730281381...
2512268    https://www.kickstarter.com/projects/louisvill...
Name: project_url, Length: 2321752, dtype: object

In [13]:
# Drop extra columns
df.drop(['creator', 'id', 'pledged', 'category', 'urls', 'converted_pledged_amount', 'source_url'], axis=1, inplace=True)

In [14]:
# Rename category column
df.rename(columns={'new_category': 'category'}, inplace=True)

In [15]:
df.shape

(2321752, 15)

In [16]:
df.reset_index(drop=True, inplace=True)

## Create time-based columns

In [17]:
# "Project lifetime" --> creation date to funding deadline (rounded to the nearest day)
df['project_life'] = (df['deadline'] - df['created_at']).dt.days
df['project_life'] = round(df['project_life'], 0)

# Length of the campaign (rounded to the nearest day)
df['campaign_length'] = (df['deadline'] - df['launched_at']).dt.days
df['campaign_length'] = round(df['campaign_length'], 0)

# Year and month of launch
df['launch_month'] = pd.to_datetime(df['launched_at']).dt.to_period('M')

# year and month of deadline
df['deadline_month'] = pd.to_datetime(df['deadline']).dt.to_period('M')

## Aggregate monthly data to get averages and totals

In [18]:
df.head(5)

,backers_count,country,created_at,deadline,goal,launched_at,name,spotlight,staff_pick,state,state_changed_at,usd_pledged,category,subcategory,project_url,project_life,campaign_length,launch_month,deadline_month
0,1,US,2016-09-11 22:05:51,2016-12-05 19:42:23,5000.0,2016-11-05 18:42:23,Industrial Bamboo Table,False,False,failed,2016-12-05 19:42:23,240.000000,crafts,woodworking,https://www.kickstarter.com/projects/983022919...,84,30,2016-11,2016-12
1,3,CA,2015-07-10 14:59:32,2015-08-24 12:00:34,1000.0,2015-07-21 12:00:34,"Custom Pet Portraits on Canvas- ""From Pixels t...",False,False,failed,2015-08-24 12:00:34,247.950175,art,painting,https://www.kickstarter.com/projects/101531536...,44,34,2015-07,2015-08
2,243,US,2015-03-24 17:41:14,2015-05-15 16:22:34,35000.0,2015-04-15 16:22:34,The Barmobile: Boston's Mobile Cocktail Cateri...,True,True,successful,2015-05-15 16:22:34,41738.000000,food,food trucks,https://www.kickstarter.com/projects/372111659...,51,30,2015-04,2015-05
3,27,US,2017-05-18 12:30:32,2017-07-16 15:03:03,3000.0,2017-06-01 15:03:03,Grandfather Thunder & The Night Horses,True,False,successful,2017-07-16 15:03:04,3115.000000,publishing,children's books,https://www.kickstarter.com/projects/133655246...,59,45,2017-06,2017-07
4,3,US,2017-06-11 02:02:05,2017-09-09 19:17:10,250000.0,2017-08-10 19:17:10,Parker's Cup Coffee & Tea,False,False,failed,2017-09-09 19:17:10,61.000000,food,drinks,https://www.kickstarter.com/projects/107038669...,90,30,2017-08,2017-09


In [19]:
# Group the data by name and find the latest data (maximum pledge amounts)
max_amounts = df.groupby('name', as_index=False).usd_pledged.max()

In [20]:
# Inner join the data to the original dataset to filter for unique projects
kickstarter_df = pd.merge(df, max_amounts, how='inner', left_on=['name', 'usd_pledged'], right_on=['name', 'usd_pledged'])
kickstarter_df.drop_duplicates('name', inplace=True)

In [21]:
print(kickstarter_df.shape)
kickstarter_df.head(5)

(194318, 19)


,backers_count,country,created_at,deadline,goal,launched_at,name,spotlight,staff_pick,state,state_changed_at,usd_pledged,category,subcategory,project_url,project_life,campaign_length,launch_month,deadline_month
0,1,US,2016-09-11 22:05:51,2016-12-05 19:42:23,5000.0,2016-11-05 18:42:23,Industrial Bamboo Table,False,False,failed,2016-12-05 19:42:23,240.000000,crafts,woodworking,https://www.kickstarter.com/projects/983022919...,84,30,2016-11,2016-12
12,3,CA,2015-07-10 14:59:32,2015-08-24 12:00:34,1000.0,2015-07-21 12:00:34,"Custom Pet Portraits on Canvas- ""From Pixels t...",False,False,failed,2015-08-24 12:00:34,247.950175,art,painting,https://www.kickstarter.com/projects/101531536...,44,34,2015-07,2015-08
24,243,US,2015-03-24 17:41:14,2015-05-15 16:22:34,35000.0,2015-04-15 16:22:34,The Barmobile: Boston's Mobile Cocktail Cateri...,True,True,successful,2015-05-15 16:22:34,41738.000000,food,food trucks,https://www.kickstarter.com/projects/372111659...,51,30,2015-04,2015-05
36,27,US,2017-05-18 12:30:32,2017-07-16 15:03:03,3000.0,2017-06-01 15:03:03,Grandfather Thunder & The Night Horses,True,False,successful,2017-07-16 15:03:04,3115.000000,publishing,children's books,https://www.kickstarter.com/projects/133655246...,59,45,2017-06,2017-07
48,3,US,2017-06-11 02:02:05,2017-09-09 19:17:10,250000.0,2017-08-10 19:17:10,Parker's Cup Coffee & Tea,False,False,failed,2017-09-09 19:17:10,61.000000,food,drinks,https://www.kickstarter.com/projects/107038669...,90,30,2017-08,2017-09


In [26]:
# Calculate daily backers and funding
kickstarter_df['daily_amount_pledged'] = round(kickstarter_df['usd_pledged'] / kickstarter_df['campaign_length'], 0)
kickstarter_df['daily_backers'] = round(kickstarter_df['backers_count'] / kickstarter_df['campaign_length'], 4)

In [27]:
print(kickstarter_df.daily_amount_pledged.min(), kickstarter_df.daily_amount_pledged.max())
print(kickstarter_df.daily_backers.min(), kickstarter_df.daily_backers.max())

0.0 332646.0
0.0 3052.8333


In [29]:
kickstarter_df.reset_index(drop=True, inplace=True)
print(kickstarter_df.shape)
kickstarter_df.head()

(194318, 21)


,backers_count,country,created_at,deadline,goal,launched_at,name,spotlight,staff_pick,state,...,usd_pledged,category,subcategory,project_url,project_life,campaign_length,launch_month,deadline_month,daily_amount_pledged,daily_backers
0,1,US,2016-09-11 22:05:51,2016-12-05 19:42:23,5000.0,2016-11-05 18:42:23,Industrial Bamboo Table,False,False,failed,...,240.000000,crafts,woodworking,https://www.kickstarter.com/projects/983022919...,84,30,2016-11,2016-12,8.0,0.0333
1,3,CA,2015-07-10 14:59:32,2015-08-24 12:00:34,1000.0,2015-07-21 12:00:34,"Custom Pet Portraits on Canvas- ""From Pixels t...",False,False,failed,...,247.950175,art,painting,https://www.kickstarter.com/projects/101531536...,44,34,2015-07,2015-08,7.0,0.0882
2,243,US,2015-03-24 17:41:14,2015-05-15 16:22:34,35000.0,2015-04-15 16:22:34,The Barmobile: Boston's Mobile Cocktail Cateri...,True,True,successful,...,41738.000000,food,food trucks,https://www.kickstarter.com/projects/372111659...,51,30,2015-04,2015-05,1391.0,8.1000
3,27,US,2017-05-18 12:30:32,2017-07-16 15:03:03,3000.0,2017-06-01 15:03:03,Grandfather Thunder & The Night Horses,True,False,successful,...,3115.000000,publishing,children's books,https://www.kickstarter.com/projects/133655246...,59,45,2017-06,2017-07,69.0,0.6000
4,3,US,2017-06-11 02:02:05,2017-09-09 19:17:10,250000.0,2017-08-10 19:17:10,Parker's Cup Coffee & Tea,False,False,failed,...,61.000000,food,drinks,https://www.kickstarter.com/projects/107038669...,90,30,2017-08,2017-09,2.0,0.1000


## Save DataFrame to pickle File

In [30]:
# Save dataframe to pickle file
with open('kickstarter_datafull.pickle', 'wb') as outfile:
    pickle.dump(kickstarter_df, outfile)